## `lab06`—The Enigma Machine

![](./img/lab04-header-bkgd.png)

❖ Objectives

-   Understand basic concepts of remapping sets.
-   Build a transformation pipeline of simple elements.
-   Prepare a code implementation of a cipher.

### The Caesar Cipher

In today's lab, we will explore some fundamentals of cryptography to implement a simplified version of the World War II-era Nazi Enigma machine (most recently featured in the 2014 film *The Imitation Game* about Alan Turing).

In this lab we'll discuss the *Caesar cipher*, in which each letter of the alphabet is mapped forward to obscure a message.  Let's recreate that code and then extend it to use an arbitrary offset for the encoding.

![](./img/caesar-base.png)

There was a straightforward way to implement such a cipher.  Use two strings of the alphabet, one offset by one character, and build an encoding dictionary `e` to produce a message `encoded`.

In [2]:
# Set up encoding dictionary
alphabet1 = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
alphabet2 = 'BCDEFGHIJKLMNOPQRSTUVWXYZA'
e = { }
for i in range( len( alphabet1 ) ):
    e[ alphabet1[ i ] ] = alphabet2[ i ]

# Set up message to be encoded
message = "HELLO"
message = message.upper()

# Encode message
encoded = ''
for c in message:
    if c in alphabet1:
        encoded += e[ c ]
    else:
        encoded += c

print( '%s encoded becomes %s' % ( message,encoded ) )

HELLO encoded becomes IFMMP


### <span style="color:#345995">Exercise 1: Encode Caesar </span>


-   Compose a function `encode_caesar` which implements the identical functionality:  it should accept a variable `message`, transform it according to the Caesar cipher above, and `return` a variable `encoded`.

In [10]:
#grade
# define your function here
def encode_caesar( message ):
    # Set up the encoding dictionary
    alphabet1 = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    alphabet2 = 'BCDEFGHIJKLMNOPQRSTUVWXYZA'
    e = { }

    ## YOU WRITE THIS BLOCK (USE THE ABOVE CODE IF YOU NEED HELP)
    ## Step 1: set up the dictionary e with mappings from alphabet1 to alphabet2
    for i in range( len( alphabet1 ) ):
        e[ alphabet1[ i ] ] = alphabet2[ i ]
    ## Step 2: encode message
    message = message.upper()

# Encode message
    encoded = ''
    for c in message:
        if c in alphabet1:
            encoded += e[ c ]
        else:
            encoded += c
    # Finally, return the encoded message.
    return encoded

In [11]:
# test your code here.  You may edit this cell, and you may use any sample text, but the following is provided for
# convenience.
text = """The Adventures of Sherlock Holmes, by Arthur Conan Doyle"""
encode_caesar( text )

'UIF BEWFOUVSFT PG TIFSMPDL IPMNFT, CZ BSUIVS DPOBO EPZMF'

In [12]:
# it should pass this test---do NOT edit this cell
test_text = """When I glance over my notes and records of the Sherlock Holmes cases between the years '82 and '90, I am faced by so many
which present strange and interesting features that it is no easy matter to know which to choose and which to leave."""
code_text = """XIFO J HMBODF PWFS NZ OPUFT BOE SFDPSET PG UIF TIFSMPDL IPMNFT DBTFT CFUXFFO UIF ZFBST '82 BOE '90, J BN GBDFE CZ TP NBOZ
XIJDI QSFTFOU TUSBOHF BOE JOUFSFTUJOH GFBUVSFT UIBU JU JT OP FBTZ NBUUFS UP LOPX XIJDI UP DIPPTF BOE XIJDI UP MFBWF."""
result_text = encode_caesar( test_text )
assert result_text == code_text
print( 'Success!' )

Success!


This one-letter shift is easy enough to crack—you can even do it in your head for short messages.  What if we generalized it to use an `offset` (between 0 and 25, where 0 yields the same result as the original message)?  Then we could encode and decode using the same function (by offsetting forward $n$ letters, and then offsetting forward $26-n$ letters).

First, to simplify all of our later work, let's define the alphabet:

<div class="alert alert-danger">
Run this cell!
</div>

In [17]:
#grade -- for evaluation purpose, don't edit
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

To generalize our process, we need to have a general way of moving $n$ characters forward in the alphabet (and wrapping around if we count too far).

We need two tools:
-   A way to convert a character to a number (the location in the alphabet):  `alphabet.index( letter )`
-   A way to wrap around if `index + offset` is greater than the length of the alphabet (26 letters, so index 25):  the modulus or remainder operator, `%`, is well-suited for this.

Here we encode the letter `'A'` using an offset of `5`:

In [18]:
letter = 'A'  # letter to encode
offset = 5    # offset
index = alphabet.index( letter )
shifted_index = ( index + offset ) % 26

print( alphabet[ shifted_index ] )

F


Here we encode the letter `'Y'` using an offset of `20`:

In [19]:
letter = 'Y'  # letter to encode
offset = 20    # offset
index = alphabet.index( letter )
shifted_index = ( index + offset ) % 26

print( alphabet[ shifted_index ] )

S


# <span style="color:#345995">Exercise 2: Mapper </span>


-   Compose a function `mapper` which accepts a single-character string letter and an offset and `returns` the letter index mapped forward by the offset.  The periodic nature of the alphabet should be accounted for—that is, `'Z'` offset by `2` should `return` `'B'`.  The default value of `offset` should be `1`; this should be a _default argument_ in the function header.

In [30]:
#grade
# define your function here
def mapper( letter,offset=1 ):
    ## YOU WRITE THIS BLOCK (USE THE ABOVE CODE IF YOU NEED HELP)
    index = alphabet.index( letter )
    shifted_index = ( index + offset ) % 26
    return alphabet[ shifted_index ] # replace this with return ____

In [31]:
# test your code here.  You may edit this cell, and you may use any sample text, but the following is provided for
# convenience.
letter = "Q"
n = 4
mapper( letter,n )

'U'

In [32]:
# it should pass this test---do NOT edit this cell
assert mapper( 'A' ) == 'B'
assert mapper( 'A',24 ) == 'Y'
print( 'Success!' )

Success!


### <span style="color:#345995">Exercise 3: Caesar Cipher </span>


With `mapper`, it is straightforward to write a general function to apply a Caesar cipher across any number of letters.

Although there are a few ways to accomplish this, most students have found it more straightforward to use `mapper` directly on each letter.  (This is more efficient for short messages.  You could also ese `mapper` to prepare a dictionary mapping each letter, and then use the `dict` to map each letter as it is encoded.  This is more efficient for longer messages, but seems to be too involved for the length of a lab.)

-   Compose a function `caesar_cipher` which accepts a string `message` and an integer `offset`.  It should `return` `message` with each letter mapped by `offset` to another letter in the alphabet.  `offset` should default to `1` in the case that the user does not supply it.

In [41]:
#grade
# define your function here
def caesar_cipher(message,offset=1):
    # Set up the encoding dictionary if desired (or use mapper directly below)
    ## (Optional) YOU WRITE THIS BLOCK (USE THE ABOVE CODE IF YOU NEED HELP)
    message = message.upper()
    # Encode the message (converted to upper case)
    ## YOU WRITE THIS BLOCK (USE THE ABOVE CODE IF YOU NEED HELP)
    ## Step 1: Convert to upper case
    ## Step 2: Encode message and add to a variable encoded
    encoded = ''
    
    for c in message:
        if c in alphabet:
            encoded += mapper(c, offset)
        else:
            encoded += c        
    # Finally, return the encoded message.
    return encoded

In [42]:
# test your code here.  You may edit this cell, and you may use any sample text, but the following is provided for
# convenience.
text = """The Sign of Fear, by Arthur Conan Doyle"""
caesar_cipher(text)

'UIF TJHO PG GFBS, CZ BSUIVS DPOBO EPZMF'

In [43]:
# it should pass this test---do NOT edit this cell
# test case with specified offset
test_text = """A man should keep his little brain-attic stocked with all the furniture that he is likely to use, and the
rest he can put away in the lumber-room of his library, where he can get it if he wants it."""
code_text = """F RFS XMTZQI PJJU MNX QNYYQJ GWFNS-FYYNH XYTHPJI BNYM FQQ YMJ KZWSNYZWJ YMFY MJ NX QNPJQD YT ZXJ, FSI YMJ
WJXY MJ HFS UZY FBFD NS YMJ QZRGJW-WTTR TK MNX QNGWFWD, BMJWJ MJ HFS LJY NY NK MJ BFSYX NY."""
result_text = caesar_cipher( test_text,5 )
assert result_text == code_text
print( 'Success!' )

Success!


In [44]:
# it should pass this test---do NOT edit this cell
# test default case
test_text = """You know my methods, Watson."""
code_text = """ZPV LOPX NZ NFUIPET, XBUTPO."""
result_text = caesar_cipher( test_text )
assert result_text == code_text
print( 'Success!' )

Success!


### <span style="color:#345995">Exercise 4: Rotor Ciphers </span>


`rot13` is [a common encoding used online](https://infogalactic.com/info/ROT13) to simply hide the answers to questions—the digital equivalent of printing the answer upside down in the corner.  `rot13` simply rotates the alphabet thirteen characters over—a Caesar cipher of half the alphabet's length.

You can easily write a code to `rot13` a text now:

In [45]:
def rot13( message ):
    return caesar_cipher( message,offset=13 )

Conveniently, a `rot13`-ed message is its own inverse:

In [46]:
message = "elementary"
print( message )
print( rot13( message ) )
print( rot13( rot13( message ) ) )

elementary
RYRZRAGNEL
ELEMENTARY


### Rotor Cipher Machines

A straightforward extension of the basic substitution cipher (what we've referred to as the Caesar cipher) is the *polyalphabetic cipher*, in which each letter pressed on a mechanical keyboard would both be encoded and trigger a rotor to change position.  Since the rotor determines the alphabet (either the offset or a randomized substitution pattern), each letter press *changes the subsequent encoding alphabet*.  It's rather like changing the offset according to some rule every time you encoded a letter above.

To clarify this, first think of a pair of rotors, or wheels.  The inner (red) wheel represents the base alphabet (of the message), and the outer (blue) wheel represents the letter each inner letter maps to (the encoding).  An offset of one produces the following diagram:

![](./img/rotor-base.png)

A rotor cipher simply chains multiple wheels together, so that a change in one wheel produces an encoded letter *but also changes the position of the encoding rotor* for the next letter.  For instance, before encoding the letter `'A'` from the inner wheel, the rotor configuration is at left.  After encoding `'A'` (to `'B'`), the wheel advances and gives us the *new* configuration at right, in which `'A'` now maps to `'C'`.

![](./img/rotor-pair.png)

In order to think about a rotor cipher, you will have to accept a message, and for each letter in that message you will need to:

1.  Encode the letter.
2.  Advance the offset of the rotor by some value $n$.  We will take $n = 1$ for simplicity.

-   Compose a function `rotor_cipher` which accepts a string `message` and an integer `n`, the starting offset for the cipher.  `rotor_cipher` should `return` the message transformed by successively apply the rotor cipher across the alphabet.

In [128]:
#grade
# define your function here
def rotor_cipher(message, n):
    encoded = ''
    
    # Step 1: Convert the message to upper-case.
    ## YOU WRITE THIS BLOCK
    message = message.upper()
    # Step 2: Set up initial offset
    ## YOU WRITE THIS BLOCK
    offset = n
    # Step 3: Loop over each letter of the message.
    for i in message:
        if i in alphabet:
            encoded += caesar_cipher(i,offset=n)
            n += 1
        else:
            encoded += i
        
        # If the letter is in the alphabet, then:
            # 1. encode the letter
            # 2. advance the offset by n_offset=1 (modulo 26)
        # Otherwise, just add the whitespace or punctuation or numeric character to the encoded string.

    # Finally, return the encoded message.
    return encoded

In [129]:
# test your code here.  You may edit this cell, and you may use any sample text, but the following is provided for convenience.
# As an example, 'HELLO' maps to 'HFNOS' if it starts with an offset of 0:  H->H, E->F (1), L->N (2), L->O (3), O->S (4).
text = 'HELLO'
rotor_cipher(text,0)

'HFNOS'

In [130]:
# it should pass this test---do NOT edit this cell
test_text = """The observer who has thoroughly understood one link in a series of incidents should be able to accurately state all the other
ones, both before and after."""
code_text = """TIG RFXKYDNB HTB VPI KZHLJQDFKY VPGIWYAWXN ZZR ZXDB AG U NAOGDS PH LRHOKMWDD EUCJBU TX UWHB RN ADEXVFZLTH CEMGS PBC LAY JPECQ
OOGV, FTZO JNPZDR OCT RXMYM."""
result_text = rotor_cipher(test_text,0)
assert result_text == code_text
print('Success!')

Success!


In [131]:
# it should pass this test---do NOT edit this cell
test_text = """You observed that her right glove was torn at the forefinger, but you did not apparently see that both glove and finger were
stained with violet ink. She had written in a hurry and dipped her pen too deep."""
code_text = """YPW RFXKYDNN ETNH WUI JBACP DJNVF YDW YUYV JD ETR TDHVXBHBAO, ZTT ZQX HNJ UWC KABNFTDKDR MZA QFZT CQWL LRVDN KYP SWCWVJ PYMA
PRZIOGG ANZO DRYWQG WCA. JZX BVZ TPHTUGQ MS G OCABJ MAR SYGHXX CAO NDN UQR HJKW."""
result_text = rotor_cipher(test_text,0)
assert result_text == code_text
print('Success!')

Success!


### <span style="color:#345995">Exercise 5: Two Rotor Cipher </span>


While encoding strings with a single rotor provides *some* security, it's actually easy enough to break this code with a computer since the key is only 26 characters long.  (And, in this case, in alphabetical order.)

So the next logical improvement is to randomize the order of the letters in the alphabet.  We'll skip that improvement in favor of the next:  adding a second rotor.

When this happens, the *first* rotor proceeds as we have seen before.  When it makes a complete cycle (26 characters have been encoded), then it trips the *second* rotor forward one.

That is, for offsets `1,1`, originally `'A'` maps to the first rotor at `'B'` and then to the second rotor at `'C'` for a net transformation of `'A'` to `'C'`.

![](./img/two-rotors1.png)

After encoding `'A'`, the offsets are `2,1`:  the first rotor advances (and the second stays relatively offset at one from the first rotor) such that `'A'` maps to the first rotor at `'C'` and then to the second rotor at `'D'` for a net transformation of `'A'` to `'D'`.

![](./img/two-rotors2.png)

Once things cycle back around though (26 characters), the offsets are `0,2`:  `'A'` maps to the first rotor at `'A'` and then to the second rotor at `'C'` for a net transformation of `'A'` to `'C'`.

![](./img/two-rotors3.png)

With two rotors then, the key repeats every $26 \times 26 = 676$ characters.

-   Compose a function `two_rotors` which accepts a string `message` and two integers `m` and `n`, the offsets of the first and second rotors, respectively.  `two_rotors` should `return` the `message` transformed by the two-rotor cipher method as detailed above.

In [157]:
#grade
# define your function here
def two_rotors( message,m,n ):
    encoded = ''
    # Convert the message to upper-case.
    ## YOU WRITE THIS BLOCK
    message = message.upper()
    # Set up two offsets
    ## YOU WRITE THIS BLOCK
    
    # Loop over each letter of the message.
    ## YOU WRITE THIS BLOCK
    for i in message:
        if i in alphabet:
            encoded += mapper(i,offset=(m+n))
            m += 1
            if m >= 26:
                n = n + 1
                m = 0
            if n >= 26:
                n = 0
        else:
            encoded += i

    # Finally, return the encoded message.
    return encoded

In [158]:
# test your code here.  You may edit this cell, and you may use any sample value, but the following is provided for convenience.
text = """Mr. Fowler being a persevering man, as a good seaman should be, blockaded the house, and having met you succeeded by certain
arguments, metallic or otherwise, in convincing you that your interests were the same as his."""
two_rotors(text,1,1)

'OU. JTCSMA LPUAU P FVJLYQAOGMG ODR, FY H OXYO EROBQE KAIPHA ZD, BMRGPGKMM DSQ UCJIV, SGX CWSGMG NGX DUB ADMNQRRTT SQ VYMPXGM\nASIXRKUBB, WPFNZAYT GK IODBPVITG, LR IVVESYOVBV OFM MBVP VMTR JPWIWLACC HQES IXV KTGZ WP FHS.'

In [159]:
# it should pass this test---do NOT edit this cell
test_text = """Mr. Fowler being a persevering man, as a good seaman should be, blockaded the house, and having met you succeeded by certain
arguments, metallic or otherwise, in convincing you that your interests were the same as his."""
code_text = """OU. JTCSMA LPUAU P FVJLYQAOGMG ODR, FY H OXYO EROBQE KAIPHA ZD, BMRGPGKMM DSQ UCJIV, SGX CWSGMG NGX DUB ADMNQRRTT SQ VYMPXGM
ASIXRKUBB, WPFNZAYT GK IODBPVITG, LR IVVESYOVBV OFM MBVP VMTR JPWIWLACC HQES IXV KTGZ WP FHS."""
result_text = two_rotors(test_text,1,1)
assert result_text == code_text
print('Success!')

Success!


In [160]:
#Test--
#grade
def two_rotors_decode( message,m,n ):
    encoded = ''
    message = message.upper()
    
    for letter in message:
        if letter in alphabet:
            encoded += mapper( letter,n+m )
            n = ( n-1 ) % 26  # the only differences to decode
            if n % 26 == 0:
                m = ( m - 1 ) % 26  # here as well
        else:
            encoded += letter
    
    # Finally, return the result.
    return encoded

code_text = """OU. JTCSMA LPUAU P FVJLYQAOGMG ODR, FY H OXYO EROBQE KAIPHA ZD, BMRGPGKMM DSQ UCJIV, SGX CWSGMG NGX DUB ADMNQRRTT SQ VYMPXGM
ASIXRKUBB, WPFNZAYT GK IODBPVITG, LR IVVESYOVBV OFM MBVP VMTR JPWIWLACC HQES IXV KTGZ WP FHS."""
print( two_rotors_decode( code_text,1,1 ) )

QU. IRZOHU EHLQJ D SHUVHYHULQJ QEQ, DV D JRRG VHDPDQ VKRXOG EH, EOSGNDGHG WKH KRXVH, DQG KDYLQJ PHX CRX VXFFHHGHG EB FHUWDLQ
DUJXQIQWV, PHWDOOLF RU RWKHUZLVH, LQ GSQYLQFLQJ BRX WKDW BRXU LQWHUIWWV ZHUH WKH VDPH DV KLV.


In [161]:
print( two_rotors_decode( result_text,25,25 ) )

MR. FOWLER BEING A PERSEVERING MAN, AS A GOOD SEAMAN SHOULD BE, BLOCKADED THE HOUSE, AND HAVING MET YOU SUCCEEDED BY CERTAIN
ARGUMENTS, METALLIC OR OTHERWISE, IN CONVINCING YOU THAT YOUR INTERESTS WERE THE SAME AS HIS.


### <span style="color:#345995">Just for Fun: The Enigma Machine (WWII) Encoding Security </span>


We expect this approach to be far more robust against letter frequency attacks.  Let's briefly compare the frequency of the letters in an encoded block of text.

We'll need the ability to sort a dictionary of key-value pairs into two lists (of the keys and of the values), sorted by the values, in order to plot in order of value.  We can use this, for instance, to sort the standard English letter frequencies by order of frequency.  The first block below carries out this sorting; the second plots it.  **Neither block requires modification.**

In [ ]:
# Convert a dictionary to two lists sorted by the value of the second.
#grade
import operator
def dict2sort(in_dict):
    keys = sorted(in_dict, key=in_dict.get)[::-1]
    values = []
    for key in keys:
        values.append(in_dict[key])
    return keys, values

In [ ]:
# plotting boilerplate---you'll learn about this in class
#grade
import matplotlib as mpl               # import MatPlotLib
import matplotlib.pyplot as plt        # import PyPlot
%matplotlib inline
mpl.rcParams['figure.figsize'] = 15,3  # set the aspect ratio and size of the figure

# We'll use this function to plot the letters and frequencies for the next while.
# You don't need to interpret it yet, but you can examine it as much as you like.
def plot_freq(xs, ys, title=None):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    N = len(xs)

    ## necessary variables
    import numpy as np    # the Numerical Python package---you'll see this soon in lecture
    index = np.arange(N)  # x locations for bars
    width = 0.75          # bar width

    ## the bars
    rects1 = ax.bar(index, ys, width, color='blue')

    # axes and labels
    ax.set_title(title)
    ax.set_ylabel('Proportion')

    ax.set_xlim(-width,len(index)+width)
    ax.set_ylim(0,.20)
    xTickMarks = xs
    ax.set_xticks(index+width/2)
    xtickNames = ax.set_xticklabels(xTickMarks)
    plt.setp(xtickNames, fontsize=10)
    
    plt.show()

In [ ]:
# Get standard English frequencies, sorted by order of frequency.
english_dict = {'A':0.0834,'B':0.0154,'C':0.0273,'D':0.0414,'E':0.126,'F':0.0203,'G':0.0192,'H':0.0611,'I':0.0671,'J':0.0023,'K':0.0087,
                'L':0.0424,'M':0.0253,'N':0.068,'O':0.077,'P':0.0166,'Q':0.0009,'R':0.0568,'S':0.0611,'T':0.0937,'U':0.0285,'V':0.0106,
                'W':0.0234,'X':0.002,'Y':0.0204,'Z':0.0006}
letters, freqs = dict2sort(english_dict)
plot_freq(letters, freqs, title='Expected Letter Frequency of English-Language Text')

We'll use our functions from last week to aid in this.  Recall the function `calc_freq`, which returned a `dict` containing the relative frequency of letters in a sample of text.

In [49]:
# define your function here
#grade
from string import whitespace, punctuation, digits
from string import ascii_uppercase as alphabet

def calc_freq(text):
    # Create an empty frequency dictionary letter_freq.
    letter_freq = {}
    
    # Make text upper-case.
    text = text.upper()
    
    # Loop over each letter of the alphabet:
    for letter in alphabet:
        letter_freq[letter] = text.count(letter)
    
    # Make a copy of text without non-alphabet characters.
    from string import whitespace, punctuation, digits
    for character in whitespace+punctuation+digits:
        text = text.replace(character, '')
    
    # Normalize the frequencies and put the results back into letter_freq.
    for key in letter_freq.keys():
        letter_freq[key] = letter_freq[key] / len(text)
    
    # Finally, return the dict letter_freq.
    return letter_freq

In [50]:
code = """'LAY “XKJNTTBDOI” SATJNBD YON RV VXMXNFZDD CTLIKRF IB DSQ ETQC FNGWAOQ VHPUH ICVYMBCTAAG QJ S WYXEJYK ASG FEQIBTJLWQ OM
UZFBNZ IBYMS. BNWLTANP CRP EHPYUUM IA PEGR PBRHV NY VACOYEVPAO KAY XKJNTTBDOI SATJNBD, UG WH QCEINP BOTAMNB IFYF BX NPRVBT QEV
BIRBQSIHCWI HUTXDDLNYS UKEUMCKKQ NF BP LRYKNZJV GMEWPRCW HL V OCZL PT FSRVBBJLWQ IOGYRTEY, XKKOUUCEPJ VYMMSNMGSH, QEV LI AKOSH.
UJH JZTKIVOYFNZ EHFTEYHO FLVPNYII GYM, QYHQISG, JYW LUHA FL DBEK GFYL, IWN T TNJT SYGLYI PEC BOOSYYGITN XFYOSGI WGK YSLIGBIU
WVJGAUNXE MF WCLFDOCIC QFD LFCWY IBUKBZGF HTSYFBKPA. F FNPF UKTXATH DZ SVJT QE SVWJQKR NF UJH VKSICSZZF CU JYW VIHLRRZBMG QYRIMAC,
QGAQIYFFL, UIZ PM EOSVK XT UVN KYAGVTH. KZBM REIJ HNDNXHJ G KNFPXBDBUEL HZ ODB RGEPTB SK LBVMEUBBH EW S KYVH SYQIBDOI JDWZNDERR XD
KWKGN KC ANMQWWEGRL VDWNRFH. QTUHLYEKE SO NA GIKOUQCSZA, O CKDTXL DO ZMLPVVDFQK PN RDD PSRYDSE WVJ YC VRJVWIS JVEW LJ M ZPSYAGY.
(VHXL SUSKQK, 1937)\n'"""

code_dict = calc_freq(code)
code_letters, code_freqs = dict2sort(code_dict)
plot_freq(letters, freqs, title='Expected Letter Frequency of English-Language Text')
plot_freq(code_letters, code_freqs, title='Letter Frequency in Code Text')

NameError: name 'dict2sort' is not defined

The occurrence of each letter is nearly uniform, leaving us no way to attempt to break this code by a frequency analysis!  Higher-order patterns must be identified and analyzed, which is precisely what the British Ultra project did before and during World War II.

The [Nazi Enigma machine](https://infogalactic.com/info/Enigma_machine) was used extensively to send top-secret military communications to troops and operatives in the field.  Enigma had either three or four rotors and several other mapping devices within it, which effected a fiendishly complex mathematical transformation closely related to what you've done above.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Enigma_rotor_set.png/440px-Enigma_rotor_set.png)

Famously, the code was cracked for the first time in 1939 by British mathematicians, including Alan Turing, one of the fathers of computer science.  The British kept this codebreaking secret and successfully used it to counter Nazi assaults during the war.  (A similar process took place against the [Imperial Japanese PURPLE machine](https://infogalactic.com/info/Purple_(cipher_machine)) by the American government.)  This code breaking was heavily fictionalized in the novel *Cryptonomicon* by Neal Stephenson.

We will reproduce a computer-based solution of the Enigma cipher during a later lab, `decode`.

Cryptography has grown vastly more sophisticated from these early efforts, but it is still rooted in the idea of a set of transformations applied either to letters directly or to the binary representations those characters have on the machine.